In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/Shareddrives/FP/FP/ML_ImageCaptioning/

## Setting up conda

This part is crucial as google colab does not have conda by default

In [ ]:
%env PYTHONPATH =
!wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
!chmod +x Miniconda3-py38_4.12.0-Linux-x86_64.sh
!./Miniconda3-py38_4.12.0-Linux-x86_64.sh -b -f -p /usr/local
#!conda update conda
!conda update -n base -c defaults conda
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

In [ ]:
#%%shell
!conda create -n ICwFR python=3.9
!conda activate ICwFR
!conda --version

## Installing requirements

In [ ]:
!pip install torch torchvision  --index-url https://download.pytorch.org/whl/cu121
!pip  install -r requirements.txt
!python -m spacy download en
!python grit/models/ops/setup.py build develop

# Inputting images and faces into the database
Note: Please change the path into your own image path

In [ ]:
!python cap.py --face_input "/content/drive/Shareddrives/FP/FP/Captioning/images/0001.jpg" --name_input "Owen"

# Caption Output


| Arguments | Purpose |  
| -- | -- |
| `--image_input` | the directory path to the input image |
| `--no_facial_recognition` | output without face recognition |
| `--tts_path` | the directory path to the tts output |
| `--output_path` | the directory path to the .txt caption |
| `--exp_checkpoint` | the file path to the pre-trained weights |

In [ ]:
!python cap.py --image_input "/content/drive/Shareddrives/FP/FP/Captioning/images/0003.jpg" --exp_checkpoint "/content/drive/Shareddrives/FP/FP/Captioning/grit/grit_checkpoint_4ds.pth"